# Scraping - subtítulos YIFY

Este Notebook scrapea subtítulos de YIFI, guarda los archivos, carga el texto como variable de un dataframe, quita duplicados. Pasos:

1. [Descarga subtítulos](#download)
2. [Carga de subtítulos como variable de texto](#upload)
3. [Duplicados](#dup)


In [ ]:
# Importamos librerías 

## Módulos generales
from libraries import *

In [ ]:
# Completar con directorios 
gitwd = ""
datawd = ""

<a id='download'></a>
## Descarga subtítulos

In [ ]:
# Listado de películas a buscar
titles = pd.read_csv(datawd + "/titles_restricted.csv")

In [ ]:
os.chdir(datawd)
files = os.listdir()

if 'titles_subt.csv' in files: # ya habíamos comenzado la descarga y se interrumpió
    titles_subt = pd.read_csv('titles_subt.csv')
    urls = titles_subt.urls
    ns = list(titles_subt.index[titles_subt.subt.isnull()]) # el primer nulo en subt es donde dejamos

if 'titles_subt.csv' not in files:
    titles_subt = titles[["tconst"]]
    titles_subt.loc[:,"subt"] = "" # guardar nombre de archivos de subtítulos o "no tiene" si no encontramos subtítulos
    urls = ['https://yts-subs.com/movie-imdb/'+t for t in titles.tconst] 
    titles_subt.loc[:,"urls"] = urls # iterar por las páginas de subtítulos y descargarlos
    ns = list(titles_subt.index) # desde el inicio

    
# subtítulos ya descargados
os.chdir(datawd+"/subts")
already_downloaded = os.listdir()

# cada 1000 guardamos una copia del dataset
time_to_save = list(np.arange(1000, len(urls), 1000))
print(len(time_to_save))

# deshabilitar SettingWithCopyWarning 
pd.options.mode.chained_assignment = None  # default='warn'
print(ns[:5])

# loop
for n in tqdm(ns):
    
    r = requests.get(urls[n])  
    html = r.content.decode('utf-8')
    bhtml = bs4.BeautifulSoup(html) 
    links = [d.get('href') for d in bhtml.find_all('a',attrs = {'class':"subtitle-download"})]
   
    if not links: # está vacía la lista de links (no tiene subtítulo)
        titles_subt.loc[n, "subt"] = 'no tiene'
    
    else:
        try:
            english = [not not re.search('english',l) for l in links]
            link = np.array(links)[english][0].split('/')[-1]
            
            descargar = 'https://yifysubtitles.org/subtitle/' + link + '.zip' # el link del zip para la descarga empieza en https://yifysubtitles.org/subtitle y termina en .zip
            request = requests.get(descargar)
            file = zipfile.ZipFile(BytesIO(request.content)) # archivo a descargar
            
            name = titles.tconst[n] +  file.namelist()[0][-4:] # nombre de archivo a usar
            titles_subt.loc[n, "subt"] = name

            if name not in already_downloaded: # descargar aquellos que no se hayan descargado todavía
                file.extract(member = file.namelist()[0], path = sdir + "/temp" ) # lo descarga al path (carpeta temporaria con otro nombre)
                os.rename(sdir + "/temp/" + file.namelist()[0], 
                          sdir + "/" + name) # renombrar luego de guardar y mover a la carpeta permanente
                os.rmdir( sdir + "/temp" ) # eliminar carpeta temporaria
                
        except:
            titles_subt.loc[n, "subt"] = 'no tiene' # a veces dice 'no se encuentra el link en nuestros servidores'

    if n in time_to_save: # every 1000 subts, lets save the dataset
        titles_subt.to_csv(datawd + "/titles_master.csv", index = False)

# when finished, save
titles_subt.to_csv(datawd + "/titles_master.csv", index = False)      



<a id='upload'></a>
## Películas con subtítulos y duplicados

In [ ]:
print("Porcentaje de películas sin subtítulos: ",
      np.mean(titles_subt.subt == "no tiene") * 100)

master_subt_content = titles_subt[titles_subt.subt != "no tiene"].reset_index(drop = True)

print("Nombres de archivo de subtítulos duplicados:",np.sum(master_subt_content.subt.duplicated()))

In [ ]:
# Variable con los nombres completos de los archivos (incluyendo directorio)
master_subt_content["files"] = [datawd + "/subts/" + master_subt_content.subt[i] for i in range(master_subt_content.shape[0])]

# Leer contenido para cada película 
subtitulos = []

for f in tqdm(range(master_subt_content.shape[0])):
    try:
        subtitulos.append(' '.join(open(master_subt_content.files[f], "r").readlines()))
    except: 
        try:
            subtitulos.append(' '.join(open(master_subt_content.files[f], "r", encoding='utf-8').readlines()))
        except:
            try:
                subtitulos.append(json.load(' '.join(open(master_subt_content.files[f], "r", encoding='utf-8'))))
            except:
                subtitulos.append('no abre')
                
master_subt_content["s"] = subtitulos 

print("No abre el archivo de subtítulos:", np.sum(master_subt_content.s == "no abre"))
master_subt_content = master_subt_content[master_subt_content.s != "no abre"].reset_index(drop = True)
print(master_subt_content.shape) 

# Guardar
master_subt_content.to_pickle(datawd + "/master_subt_content.pkl")     

<a id='dup'></a>
## Duplicados 

Hay casos donde tenemos duplicados por errores en YIFI, donde la película se llama muy parecido, y cargan el mismo archivo de subtítulos para dos películas distintas. Los descartamos por no saber a qué película corresponden realmente.

In [ ]:
master_subt_content["dup"] = master_subt_content.s.duplicated(keep = False)
print(f"Cantidad de subtítulos con contenido duplicado: {np.sum(master_subt_content.dup)}")

dups = set(master_subt_content.tconst[master_subt_content.dup])
master_subt_content = master_subt_content[~master_subt_content.dup].reset_index(drop = True)

# Guardar nuevamente
master_subt_content.to_pickle(datawd + "/master_subt_content.pkl")     


# agregar indicación de duplicados en el master para saber que no debemos contarslos
master.subt[master.tconst.isin(dups)] = "dup"

# save again
master.to_csv(datawd + "/titles_master.csv", index = False)